In [1]:
import torch
from torch import nn
import utils

In [2]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels,
                                kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

In [3]:
# 超参数变量conv_arch。该变量指定了每个VGG块里卷积层个数和输出通道数。全连接模块则与AlexNet中的相同。
# 原始VGG网络有5个卷积块，其中前两个块各有一个卷积层，后三个块各包含两个卷积层。 
# 第一个模块有64个输出通道，每个后续模块将输出通道数量翻倍，直到该数字达到512。
# 由于该网络使用8个卷积层和3个全连接层，因此它通常被称为VGG-11。

conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [4]:
def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    # 卷积层部分
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_blks, 
        nn.Flatten(),
        # 全连接层部分
        nn.Linear(in_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))

net = vgg(conv_arch)

In [5]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__,'output shape:\t',X.shape)

Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])


In [6]:
# 由于VGG-11比AlexNet计算量更大，因此我们构建了一个通道数较少的网络，足够用于训练Fashion-MNIST数据集。
# conv_arch中每个vgg块通道数为原来的1/8
ratio = 8
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [7]:
lr, num_epochs, batch_size = 0.05, 10, 64
train_iter, test_iter = utils.load_data_fashion_mnist(batch_size, resize=224)
utils.train_ch6(net, train_iter, test_iter, num_epochs, lr, utils.try_gpu())

training on: [cuda:0], [2024-05-30 22:49:33]
epoch: 1/10, loss 0.752, train acc 0.720, test acc 0.864
epoch: 2/10, loss 0.347, train acc 0.873, test acc 0.882
epoch: 3/10, loss 0.284, train acc 0.896, test acc 0.896
epoch: 4/10, loss 0.250, train acc 0.908, test acc 0.907
epoch: 5/10, loss 0.224, train acc 0.919, test acc 0.907
epoch: 6/10, loss 0.202, train acc 0.926, test acc 0.917
epoch: 7/10, loss 0.185, train acc 0.933, test acc 0.917
epoch: 8/10, loss 0.171, train acc 0.937, test acc 0.917
epoch: 9/10, loss 0.156, train acc 0.942, test acc 0.924
epoch: 10/10, loss 0.142, train acc 0.948, test acc 0.927
*** 610.6 examples/sec on cuda:0 - [0:00:16:22], all: [0:00:21:14] ***
